# Test-00
Ejecutado en **Google Collaboratory**.

## Descripción
Se probaron seis modelos de red neuronal dentro de los cuales se fueron haciendo algunas mejoras, entre ellas:
* Agregar capas de BatchNormalization a las capas Densas de la red neuronal posterior a las capas convolucionales
* Probar con AveragePooling2D, además de MaxPooling2D
* Agregar capas de Dropout para reducir el overfitting

In [128]:
import numpy as np

In [129]:
import pandas as pd

## Cargando las bases de datos

In [130]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [131]:
x_train_valid = np.load('drive/Shareddrives/Redes Neuronales BIO/CIFAR-100/x_train.npy')
y_train_valid = np.load('drive/Shareddrives/Redes Neuronales BIO/CIFAR-100/y_train.npy')
x_test = np.load('drive/Shareddrives/Redes Neuronales BIO/CIFAR-100/x_test.npy')

# Separando conjuntos para entrenamiento, validación

In [132]:
from sklearn.model_selection import train_test_split

In [133]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train_valid, y_train_valid, test_size=0.2, random_state=15, stratify=y_train_valid)

# Normalización de datos

In [134]:
x_train_norm = x_train / 255
x_valid_norm = x_valid / 255
x_test_norm = x_test / 255

# Modelos

In [206]:
from tensorflow.keras.layers import Dense, Flatten, Activation, BatchNormalization, Dropout, Conv2D, MaxPooling2D, InputLayer, AveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow import keras

## Modelo #1

In [136]:
# Create the model
model = Sequential()
model.add(InputLayer(input_shape=(32, 32, 3)))
model.add(Conv2D(64, 3))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D())
model.add(Conv2D(128, 3))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dense(100))
model.add(Activation('softmax'))

# Compile
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy']
             )

In [137]:
model.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_42 (Conv2D)           (None, 30, 30, 64)        1792      
_________________________________________________________________
batch_normalization_76 (Batc (None, 30, 30, 64)        256       
_________________________________________________________________
activation_82 (Activation)   (None, 30, 30, 64)        0         
_________________________________________________________________
max_pooling2d_42 (MaxPooling (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_43 (Conv2D)           (None, 13, 13, 128)       73856     
_________________________________________________________________
batch_normalization_77 (Batc (None, 13, 13, 128)       512       
_________________________________________________________________
activation_83 (Activation)   (None, 13, 13, 128)     

In [138]:
# Create the ModelCheckpoint callback to save the best model during training
mc_callback = ModelCheckpoint('model_1.hdf5',
                              monitor='val_accuracy',
                              save_best_only=True,
                              verbose=0,
                              mode='max'
                             )

# Train the model
model.fit(x_train_norm, y_train,
          validation_data=(x_valid_norm, y_valid),
          callbacks=[mc_callback],
          epochs=10,
          batch_size=256
         )

Epoch 1/10
157/157 [==============================] - 4s 23ms/step - loss: 3.8962 - accuracy: 0.1173 - val_loss: 4.9996 - val_accuracy: 0.0181
Epoch 2/10
157/157 [==============================] - 3s 22ms/step - loss: 3.0688 - accuracy: 0.2491 - val_loss: 5.3394 - val_accuracy: 0.0281
Epoch 3/10
157/157 [==============================] - 3s 22ms/step - loss: 2.6574 - accuracy: 0.3281 - val_loss: 3.7739 - val_accuracy: 0.1462
Epoch 4/10
157/157 [==============================] - 3s 22ms/step - loss: 2.3971 - accuracy: 0.3851 - val_loss: 3.0715 - val_accuracy: 0.2502
Epoch 5/10
157/157 [==============================] - 3s 22ms/step - loss: 2.2185 - accuracy: 0.4227 - val_loss: 2.8276 - val_accuracy: 0.3057
Epoch 6/10
157/157 [==============================] - 3s 22ms/step - loss: 2.0887 - accuracy: 0.4518 - val_loss: 2.5625 - val_accuracy: 0.3580
Epoch 7/10
157/157 [==============================] - 3s 22ms/step - loss: 1.9713 - accuracy: 0.4780 - val_loss: 3.3425 - val_accuracy: 0.2531

In [139]:
# Load the model and show the final metrics
model = keras.models.load_model('model_1.hdf5')

# Train and validation metrics
_, train_acc = model.evaluate(x_train_norm, y_train, verbose=0)
_, valid_acc = model.evaluate(x_valid_norm, y_valid, verbose=0)

# Show result
print(f'[Accuracy] Train: {round(train_acc, 3)} Valid: {round(valid_acc, 3)}')

[Accuracy] Train: 0.51 Valid: 0.371


## Modelo #2

In [140]:
# Create the model
model = Sequential()
model.add(InputLayer(input_shape=(32, 32, 3)))
model.add(Conv2D(64, 3))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D())
model.add(Conv2D(128, 3))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(100))
model.add(BatchNormalization())
model.add(Activation('softmax'))

# Compile
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy']
             )

In [141]:
model.summary()

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_44 (Conv2D)           (None, 30, 30, 64)        1792      
_________________________________________________________________
batch_normalization_78 (Batc (None, 30, 30, 64)        256       
_________________________________________________________________
activation_86 (Activation)   (None, 30, 30, 64)        0         
_________________________________________________________________
max_pooling2d_44 (MaxPooling (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 13, 13, 128)       73856     
_________________________________________________________________
batch_normalization_79 (Batc (None, 13, 13, 128)       512       
_________________________________________________________________
activation_87 (Activation)   (None, 13, 13, 128)     

In [142]:
# Create the ModelCheckpoint callback to save the best model during training
mc_callback = ModelCheckpoint('model_2.hdf5',
                              monitor='val_accuracy',
                              save_best_only=True,
                              verbose=0,
                              mode='max'
                             )

# Train the model
model.fit(x_train_norm, y_train,
          validation_data=(x_valid_norm, y_valid),
          callbacks=[mc_callback],
          epochs=10,
          batch_size=256
         )

Epoch 1/10
157/157 [==============================] - 4s 23ms/step - loss: 3.4029 - accuracy: 0.2299 - val_loss: 4.8478 - val_accuracy: 0.0164
Epoch 2/10
157/157 [==============================] - 3s 22ms/step - loss: 2.5912 - accuracy: 0.4028 - val_loss: 4.6382 - val_accuracy: 0.0526
Epoch 3/10
157/157 [==============================] - 3s 22ms/step - loss: 2.1481 - accuracy: 0.5124 - val_loss: 3.5444 - val_accuracy: 0.1911
Epoch 4/10
157/157 [==============================] - 3s 22ms/step - loss: 1.7567 - accuracy: 0.6108 - val_loss: 2.9545 - val_accuracy: 0.3035
Epoch 5/10
157/157 [==============================] - 3s 22ms/step - loss: 1.3881 - accuracy: 0.7146 - val_loss: 2.9850 - val_accuracy: 0.3072
Epoch 6/10
157/157 [==============================] - 3s 22ms/step - loss: 1.0387 - accuracy: 0.8133 - val_loss: 2.7618 - val_accuracy: 0.3588
Epoch 7/10
157/157 [==============================] - 3s 22ms/step - loss: 0.7456 - accuracy: 0.8921 - val_loss: 2.7837 - val_accuracy: 0.3586

In [143]:
# Load the model and show the final metrics
model = keras.models.load_model('model_2.hdf5')

# Train and validation metrics
_, train_acc = model.evaluate(x_train_norm, y_train, verbose=0)
_, valid_acc = model.evaluate(x_valid_norm, y_valid, verbose=0)

# Show result
print(f'[Accuracy] Train: {round(train_acc, 3)} Valid: {round(valid_acc, 3)}')

[Accuracy] Train: 0.811 Valid: 0.359


## Modelo #3

In [144]:
# Create the model
model = Sequential()
model.add(InputLayer(input_shape=(32, 32, 3)))
model.add(Conv2D(64, 3))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D())
model.add(Conv2D(128, 3))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(256))
model.add(Dropout(0.8))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(100))
model.add(BatchNormalization())
model.add(Activation('softmax'))

# Compile
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy']
             )

In [145]:
model.summary()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_46 (Conv2D)           (None, 30, 30, 64)        1792      
_________________________________________________________________
batch_normalization_82 (Batc (None, 30, 30, 64)        256       
_________________________________________________________________
activation_90 (Activation)   (None, 30, 30, 64)        0         
_________________________________________________________________
max_pooling2d_46 (MaxPooling (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 13, 13, 128)       73856     
_________________________________________________________________
batch_normalization_83 (Batc (None, 13, 13, 128)       512       
_________________________________________________________________
activation_91 (Activation)   (None, 13, 13, 128)     

In [146]:
# Create the ModelCheckpoint callback to save the best model during training
mc_callback = ModelCheckpoint('model_3.hdf5',
                              monitor='val_accuracy',
                              save_best_only=True,
                              verbose=0,
                              mode='max'
                             )

# Train the model
model.fit(x_train_norm, y_train,
          validation_data=(x_valid_norm, y_valid),
          callbacks=[mc_callback],
          epochs=50,
          batch_size=256
         )

Epoch 1/50
157/157 [==============================] - 4s 23ms/step - loss: 4.4555 - accuracy: 0.0415 - val_loss: 4.5944 - val_accuracy: 0.0218
Epoch 2/50
157/157 [==============================] - 3s 22ms/step - loss: 4.0271 - accuracy: 0.0922 - val_loss: 4.4934 - val_accuracy: 0.0443
Epoch 3/50
157/157 [==============================] - 3s 22ms/step - loss: 3.8045 - accuracy: 0.1305 - val_loss: 3.9754 - val_accuracy: 0.1023
Epoch 4/50
157/157 [==============================] - 3s 22ms/step - loss: 3.6145 - accuracy: 0.1651 - val_loss: 3.4532 - val_accuracy: 0.1874
Epoch 5/50
157/157 [==============================] - 3s 22ms/step - loss: 3.4506 - accuracy: 0.1978 - val_loss: 3.1271 - val_accuracy: 0.2454
Epoch 6/50
157/157 [==============================] - 3s 22ms/step - loss: 3.3044 - accuracy: 0.2212 - val_loss: 3.1564 - val_accuracy: 0.2441
Epoch 7/50
157/157 [==============================] - 3s 22ms/step - loss: 3.1943 - accuracy: 0.2414 - val_loss: 2.9541 - val_accuracy: 0.2643

In [147]:
# Load the model and show the final metrics
model = keras.models.load_model('model_3.hdf5')

# Train and validation metrics
_, train_acc = model.evaluate(x_train_norm, y_train, verbose=0)
_, valid_acc = model.evaluate(x_valid_norm, y_valid, verbose=0)

# Show result
print(f'[Accuracy] Train: {round(train_acc, 3)} Valid: {round(valid_acc, 3)}')

[Accuracy] Train: 0.715 Valid: 0.446


## Modelo #4

In [200]:
# Create the model
model = Sequential()
model.add(InputLayer(input_shape=(32, 32, 3)))
model.add(Conv2D(32, 3))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D())
model.add(Conv2D(64, 3))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(256))
model.add(Dropout(0.75))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(100))
model.add(BatchNormalization())
model.add(Activation('softmax'))

# Compile
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy']
             )

In [201]:
model.summary()

Model: "sequential_42"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_86 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
batch_normalization_158 (Bat (None, 30, 30, 32)        128       
_________________________________________________________________
activation_166 (Activation)  (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_86 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_87 (Conv2D)           (None, 13, 13, 64)        18496     
_________________________________________________________________
batch_normalization_159 (Bat (None, 13, 13, 64)        256       
_________________________________________________________________
activation_167 (Activation)  (None, 13, 13, 64)      

In [202]:
# Create the ModelCheckpoint callback to save the best model during training
mc_callback = ModelCheckpoint('model_4.hdf5',
                              monitor='val_accuracy',
                              save_best_only=True,
                              verbose=0,
                              mode='max'
                             )

# Train the model
model.fit(x_train_norm, y_train,
          validation_data=(x_valid_norm, y_valid),
          callbacks=[mc_callback],
          epochs=30,
          batch_size=256
         )

Epoch 1/30
157/157 [==============================] - 3s 15ms/step - loss: 4.3646 - accuracy: 0.0524 - val_loss: 4.5750 - val_accuracy: 0.0246
Epoch 2/30
157/157 [==============================] - 2s 13ms/step - loss: 3.9047 - accuracy: 0.1113 - val_loss: 4.4343 - val_accuracy: 0.0337
Epoch 3/30
157/157 [==============================] - 2s 13ms/step - loss: 3.6633 - accuracy: 0.1561 - val_loss: 3.7831 - val_accuracy: 0.1303
Epoch 4/30
157/157 [==============================] - 2s 13ms/step - loss: 3.4609 - accuracy: 0.1944 - val_loss: 3.2006 - val_accuracy: 0.2525
Epoch 5/30
157/157 [==============================] - 2s 13ms/step - loss: 3.2995 - accuracy: 0.2289 - val_loss: 3.0725 - val_accuracy: 0.2646
Epoch 6/30
157/157 [==============================] - 2s 14ms/step - loss: 3.1633 - accuracy: 0.2526 - val_loss: 3.0172 - val_accuracy: 0.2657
Epoch 7/30
157/157 [==============================] - 2s 13ms/step - loss: 3.0557 - accuracy: 0.2714 - val_loss: 2.9054 - val_accuracy: 0.2853

In [203]:
# Load the model and show the final metrics
model = keras.models.load_model('model_4.hdf5')

# Train and validation metrics
_, train_acc = model.evaluate(x_train_norm, y_train, verbose=0)
_, valid_acc = model.evaluate(x_valid_norm, y_valid, verbose=0)

# Show result
print(f'[Accuracy] Train: {round(train_acc, 3)} Valid: {round(valid_acc, 3)}')

[Accuracy] Train: 0.536 Valid: 0.391


## Modelo #5

In [207]:
# Create the model
model = Sequential()
model.add(InputLayer(input_shape=(32, 32, 3)))
model.add(Conv2D(32, 3))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(AveragePooling2D())
model.add(Conv2D(64, 3))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(AveragePooling2D())
model.add(Flatten())
model.add(Dense(256))
model.add(Dropout(0.75))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(100))
model.add(BatchNormalization())
model.add(Activation('softmax'))

# Compile
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy']
             )

In [208]:
model.summary()

Model: "sequential_44"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_89 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
batch_normalization_163 (Bat (None, 30, 30, 32)        128       
_________________________________________________________________
activation_171 (Activation)  (None, 30, 30, 32)        0         
_________________________________________________________________
average_pooling2d (AveragePo (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_90 (Conv2D)           (None, 13, 13, 64)        18496     
_________________________________________________________________
batch_normalization_164 (Bat (None, 13, 13, 64)        256       
_________________________________________________________________
activation_172 (Activation)  (None, 13, 13, 64)      

In [209]:
# Create the ModelCheckpoint callback to save the best model during training
mc_callback = ModelCheckpoint('model_5.hdf5',
                              monitor='val_accuracy',
                              save_best_only=True,
                              verbose=0,
                              mode='max'
                             )

# Train the model
model.fit(x_train_norm, y_train,
          validation_data=(x_valid_norm, y_valid),
          callbacks=[mc_callback],
          epochs=30,
          batch_size=256
         )

Epoch 1/30
157/157 [==============================] - 3s 15ms/step - loss: 4.3372 - accuracy: 0.0567 - val_loss: 4.5337 - val_accuracy: 0.0256
Epoch 2/30
157/157 [==============================] - 2s 13ms/step - loss: 3.9092 - accuracy: 0.1160 - val_loss: 4.3663 - val_accuracy: 0.0476
Epoch 3/30
157/157 [==============================] - 2s 13ms/step - loss: 3.6974 - accuracy: 0.1534 - val_loss: 3.7651 - val_accuracy: 0.1348
Epoch 4/30
157/157 [==============================] - 2s 13ms/step - loss: 3.5191 - accuracy: 0.1868 - val_loss: 3.2847 - val_accuracy: 0.2306
Epoch 5/30
157/157 [==============================] - 2s 13ms/step - loss: 3.3778 - accuracy: 0.2128 - val_loss: 3.0405 - val_accuracy: 0.2607
Epoch 6/30
157/157 [==============================] - 2s 13ms/step - loss: 3.2477 - accuracy: 0.2349 - val_loss: 3.0744 - val_accuracy: 0.2631
Epoch 7/30
157/157 [==============================] - 2s 13ms/step - loss: 3.1319 - accuracy: 0.2579 - val_loss: 3.0642 - val_accuracy: 0.2503

In [210]:
# Load the model and show the final metrics
model = keras.models.load_model('model_5.hdf5')

# Train and validation metrics
_, train_acc = model.evaluate(x_train_norm, y_train, verbose=0)
_, valid_acc = model.evaluate(x_valid_norm, y_valid, verbose=0)

# Show result
print(f'[Accuracy] Train: {round(train_acc, 3)} Valid: {round(valid_acc, 3)}')

[Accuracy] Train: 0.523 Valid: 0.413


## Modelo #6

In [211]:
# Create the model
model = Sequential()
model.add(InputLayer(input_shape=(32, 32, 3)))
model.add(Conv2D(64, 3))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(AveragePooling2D())
model.add(Conv2D(128, 3))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(AveragePooling2D())
model.add(Flatten())
model.add(Dense(256))
model.add(Dropout(0.8))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(100))
model.add(BatchNormalization())
model.add(Activation('softmax'))

# Compile
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy']
             )

In [212]:
model.summary()

Model: "sequential_45"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_91 (Conv2D)           (None, 30, 30, 64)        1792      
_________________________________________________________________
batch_normalization_167 (Bat (None, 30, 30, 64)        256       
_________________________________________________________________
activation_175 (Activation)  (None, 30, 30, 64)        0         
_________________________________________________________________
average_pooling2d_2 (Average (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_92 (Conv2D)           (None, 13, 13, 128)       73856     
_________________________________________________________________
batch_normalization_168 (Bat (None, 13, 13, 128)       512       
_________________________________________________________________
activation_176 (Activation)  (None, 13, 13, 128)     

In [213]:
# Create the ModelCheckpoint callback to save the best model during training
mc_callback = ModelCheckpoint('model_6.hdf5',
                              monitor='val_accuracy',
                              save_best_only=True,
                              verbose=0,
                              mode='max'
                             )

# Train the model
model.fit(x_train_norm, y_train,
          validation_data=(x_valid_norm, y_valid),
          callbacks=[mc_callback],
          epochs=50,
          batch_size=256
         )

Epoch 1/50
157/157 [==============================] - 4s 23ms/step - loss: 4.4469 - accuracy: 0.0424 - val_loss: 4.5760 - val_accuracy: 0.0192
Epoch 2/50
157/157 [==============================] - 3s 22ms/step - loss: 4.0226 - accuracy: 0.0933 - val_loss: 4.4868 - val_accuracy: 0.0266
Epoch 3/50
157/157 [==============================] - 3s 22ms/step - loss: 3.8140 - accuracy: 0.1297 - val_loss: 4.0421 - val_accuracy: 0.0872
Epoch 4/50
157/157 [==============================] - 3s 22ms/step - loss: 3.6372 - accuracy: 0.1629 - val_loss: 3.3953 - val_accuracy: 0.2027
Epoch 5/50
157/157 [==============================] - 3s 22ms/step - loss: 3.4747 - accuracy: 0.1942 - val_loss: 3.0838 - val_accuracy: 0.2559
Epoch 6/50
157/157 [==============================] - 3s 22ms/step - loss: 3.3367 - accuracy: 0.2168 - val_loss: 2.9516 - val_accuracy: 0.2725
Epoch 7/50
157/157 [==============================] - 3s 22ms/step - loss: 3.2162 - accuracy: 0.2412 - val_loss: 3.2602 - val_accuracy: 0.2267

In [217]:
# Load the model and show the final metrics
model = keras.models.load_model('model_6.hdf5')

# Train and validation metrics
_, train_acc = model.evaluate(x_train_norm, y_train, verbose=0)
_, valid_acc = model.evaluate(x_valid_norm, y_valid, verbose=0)

# Show result
print(f'[Accuracy] Train: {round(train_acc, 3)} Valid: {round(valid_acc, 3)}')

[Accuracy] Train: 0.651 Valid: 0.451
